# **Homework 2-1 Phoneme Classification**

* Slides: https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* Video (Chinese): https://youtu.be/PdjXnQbu2zo
* Video (English): https://youtu.be/ESRr-VCykBs


## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task,
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
# !unzip data.zip
!unzip /content/drive/MyDrive/ML2021/HW2/timit_11_v2.zip
!ls

Archive:  /content/drive/MyDrive/ML2021/HW2/timit_11_v2.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
drive  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [3]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [5]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [6]:
BATCH_SIZE = 1024  # 增加了batch_size64 -> 1024

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
# ！输入11个（包含了前后各5个，那还能shuffle吗）！：这个输入不是在程式里实现的，而是在数据集里就已经做好了（第二维是429=11*39）
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

<ipython-input-4-f6af567917fe>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = y.astype(np.int)


Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [7]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

0

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [8]:
import torch
import torch.nn as nn

# class Classifier(nn.Module):
#     def __init__(self):
#         super(Classifier, self).__init__()
#         self.layer1 = nn.Linear(429, 1024)
#         self.layer2 = nn.Linear(1024, 512)
#         self.layer3 = nn.Linear(512, 128)
#         self.out = nn.Linear(128, 39)

#         self.act_fn = nn.LeakyReLU() # Change1:sigmoid修改成LeakyReLU
#         self.net = nn.Sequential(
#             nn.Linear(429, 1024), # 1
#             nn.LeakyReLU(),
#             nn.BatchNorm1d(1024),   # Change3：加入BN
#             # nn.Dropout(0.5),

#             nn.Linear(1024, 512), # 2
#             nn.LeakyReLU(),
#             nn.BatchNorm1d(512),
#             # nn.Dropout(0.5),

#             nn.Linear(512,128), # 3
#             nn.LeakyReLU(),
#             nn.BatchNorm1d(128),
#             # nn.Dropout(0.5),

#             # nn.Linear(1024, 512), # 4
#             # nn.LeakyReLU(),
#             # nn.BatchNorm1d(512),
#             # nn.Dropout(0.5),

#             # nn.Linear(512, 128), # 5
#             # #nn.ReLU(),
#             # nn.LeakyReLU(),
#             # nn.BatchNorm1d(128),
#             # nn.Dropout(0.5),

#             nn.Linear(128, 39)  # out
#         )

#     def forward(self, x):
#         x = self.net(x)
#         return x
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()   # 丢掉了dropout
        self.net = nn.Sequential(
            nn.Linear(429, 2048), # 1

            nn.ReLU(),
            nn.BatchNorm1d(2048),
            # nn.Dropout(0.5),
            nn.Linear(2048, 2048), # 2

            nn.ReLU(),
            nn.BatchNorm1d(2048),
            # nn.Dropout(0.5),
            nn.Linear(2048,1024), # 3

            nn.ReLU(),
            nn.BatchNorm1d(1024),
            # nn.Dropout(0.5),
            nn.Linear(1024, 1024), # 4
            nn.ReLU(),

            nn.BatchNorm1d(1024),
            # nn.Dropout(0.5),
            nn.Linear(1024, 512), # 5
            nn.ReLU(),

            nn.BatchNorm1d(512),
            # nn.Dropout(0.5),
            nn.Linear(512, 128), # 6
            nn.ReLU(),

            nn.BatchNorm1d(128),
            # nn.Dropout(0.5),
            nn.Linear(128, 39)
        )

    def forward(self, x):
        x = self.net(x)
        return x

## Training

In [9]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [10]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [11]:
# fix random seed for reproducibility
same_seeds(0)

# get device
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 40               # number of training epoch
learning_rate = 0.0001       # learning rate
# weight_decays = 1e-3   # l2正则化，但是看文献说与BN冲突，就不实用了。
# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() # 包含了softmax
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rat)

DEVICE: cuda


In [12]:
# start training
epoch = 0
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward()
        optimizer.step()

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels)
                _, val_pred = torch.max(outputs, 1)

                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()  # item() function: convert a tensor to a Python scalar. Note that the tensor shouled only have a single element

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
               best_acc = val_acc
               torch.save(model.state_dict(), model_path)
               print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/040] Train Acc: 0.498559 Loss: 1.749151 | Val Acc: 0.646542 loss: 1.171921
saving model with acc 0.647
[002/040] Train Acc: 0.584781 Loss: 1.406368 | Val Acc: 0.667974 loss: 1.088104
saving model with acc 0.668
[003/040] Train Acc: 0.603636 Loss: 1.337179 | Val Acc: 0.685280 loss: 1.024988
saving model with acc 0.685
[004/040] Train Acc: 0.614037 Loss: 1.300728 | Val Acc: 0.686825 loss: 1.017097
saving model with acc 0.687
[005/040] Train Acc: 0.619573 Loss: 1.279768 | Val Acc: 0.692760 loss: 0.996984
saving model with acc 0.693
[006/040] Train Acc: 0.622564 Loss: 1.268020 | Val Acc: 0.695289 loss: 0.979337
saving model with acc 0.695
[007/040] Train Acc: 0.624944 Loss: 1.258709 | Val Acc: 0.699565 loss: 0.969710
saving model with acc 0.700
[008/040] Train Acc: 0.626466 Loss: 1.251936 | Val Acc: 0.697155 loss: 0.972163
[009/040] Train Acc: 0.628281 Loss: 1.245299 | Val Acc: 0.698671 loss: 0.969849
[010/040] Train Acc: 0.629016 Loss: 1.242025 | Val Acc: 0.699155 loss: 0.963354
[011

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [13]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [14]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [15]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

move the prediction.csv file to Google Drive

In [16]:
import shutil

# Move the prediction.csv file to Google Drive
shutil.move('prediction.csv', '/content/drive/MyDrive/ML2021/HW2/prediction_my.csv')

'/content/drive/MyDrive/ML2021/HW2/prediction.csv'